<a href="https://colab.research.google.com/github/mmtaha/Projeto_Data_Governance_Data_Quality/blob/main/Projeto_Data_Governance_Data_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from google.colab import files

# Carregar o arquivo Excel
uploaded = files.upload()

# Definir o caminho do arquivo de entrada e saída
input_file = '/content/Cópia de Account creation - Bulk creation template -cnpj.xlsx'
output_file = '/content/consulta_cnpjs_com_endereco_separado.xlsx'

# Função para consultar CNPJ na API ReceitaWS
def consultar_cnpj_receitaws(cnpj):
    url = f'https://www.receitaws.com.br/v1/cnpj/{cnpj}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Função para consultar CNPJ na API CNPJ.biz
def consultar_cnpj_cnpj_biz(cnpj):
    url = f'https://api.cnpj.biz/{cnpj}'
    response = requests.get(url, headers={"Authorization": "Bearer YOUR_API_KEY"})
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Função para consultar CNPJ na API PublicaCNPJ
def consultar_cnpj_publicacnpj(cnpj):
    url = f'https://publica.cnpj.ws/cnpj/{cnpj}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Função principal para tentar consultar o CNPJ em diferentes APIs
def consultar_cnpj(cnpj):
    dados = consultar_cnpj_receitaws(cnpj)
    if dados and dados.get('status') != 'error':
        return dados

    dados = consultar_cnpj_cnpj_biz(cnpj)
    if dados and 'status' not in dados:
        return dados

    dados = consultar_cnpj_publicacnpj(cnpj)
    if dados and 'status' not in dados:
        return dados

    return {"status": "error", "message": "CNPJ não encontrado em nenhuma API"}

# Ler o arquivo Excel e extrair CNPJs da coluna B a partir da linha 2
df = pd.read_excel(input_file, sheet_name=0, engine='openpyxl')
cnpjs = df.iloc[1:, 1].dropna().astype(str).tolist()

# Consultar informações para cada CNPJ e armazenar os resultados
resultados = []
for cnpj in cnpjs:
    dados = consultar_cnpj(cnpj)
    if dados.get('status') != 'error':
        resultado = {
            'CNPJ': cnpj,
            'Nome': dados.get('nome', dados.get('razao_social')),
            'Rua': dados.get('logradouro', dados.get('endereco', {}).get('logradouro')),
            'Número': dados.get('numero', dados.get('endereco', {}).get('numero')),
            'Bairro': dados.get('bairro', dados.get('endereco', {}).get('bairro')),
            'Cidade': dados.get('municipio', dados.get('endereco', {}).get('cidade')),
            'Estado': dados.get('uf', dados.get('endereco', {}).get('estado')),
            'País': 'Brasil',
            'CEP': dados.get('cep', dados.get('endereco', {}).get('cep')),
            'Telefone': dados.get('telefone', dados.get('contato', {}).get('telefone')),
            'Situação': dados.get('situacao')
        }
    else:
        resultado = {
            'CNPJ': cnpj,
            'Nome': None,
            'Rua': None,
            'Número': None,
            'Bairro': None,
            'Cidade': None,
            'Estado': None,
            'País': None,
            'CEP': None,
            'Telefone': None,
            'Situação': None
        }
    resultados.append(resultado)

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# Salvar os resultados em um novo arquivo Excel
df_resultados.to_excel(output_file, index=False, engine='openpyxl')

# Mostrar o DataFrame na tela
df_resultados

# Baixar o arquivo de resultados
files.download(output_file)
